In [1]:
!pip install wordfreq
!pip install symspellpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.5/125.5 kB 13.0 MB/s eta 0:00:00


In [2]:
#!pip uninstall huggingface_hub -y && pip install git+https://github.com/huggingface/huggingface_hub
#!pip install jiwer

In [3]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset, Dataset
from tqdm import tqdm
import datasets

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True,nb_workers=8)
tqdm.pandas()

import pandas as pd
from wordfreq import word_frequency ,top_n_list,get_frequency_dict

from symspellpy import SymSpell, Verbosity
from itertools import islice



from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup,
    TrainingArguments, 
    Trainer, 
    DataCollatorForSeq2Seq,
    AutoTokenizer,
    FlaxT5ForConditionalGeneration,
    AutoModelForSeq2SeqLM
 
)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [4]:
model_name = '/kaggle/input/t5-asr-corrector-masked-second-attempt/t5_asr_bn_corrector_second_attempt'
tokenizer = AutoTokenizer.from_pretrained(model_name, local_files_only=True)
model = T5ForConditionalGeneration.from_pretrained(model_name, local_files_only=True).to('cuda')

You are using a model of type mt5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.


In [5]:
def correct_grammar(input_text,num_return_sequences):
    batch = tokenizer([input_text],truncation=True,padding='max_length',max_length=64, return_tensors="pt").to(torch_device)
    translated = model.generate(**batch,max_length=64,num_beams=4, num_return_sequences=num_return_sequences, temperature=1.5)
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    return tgt_text

In [6]:
df = pd.read_parquet("/kaggle/input/benchmark-2/benchmark_2.parquet")

In [7]:
df=df[df.audio=='noisy']

In [8]:
df = df[['path','wav2vec2','sentence']]
df

,path,wav2vec2,sentence
0,/kaggle/input/denoiser-for-benchmark/raw/train...,ছানার সাথে চিনি বা গুর্মীশিয়ে ছাঁচে ফেলে সন্দে...,ছানার সাথে চিনি বা গুড় মিশিয়ে ছাঁচে ফেলে সন্দে...
1,/kaggle/input/denoiser-for-benchmark/raw/train...,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক
2,/kaggle/input/denoiser-for-benchmark/raw/train...,গড়ি মানুষ কেসে মানুষ বলি মনে করে না এবং তুচ্ছত...,গরীব মানুষকে সে মানুষ বলেই মনে করে না এবং তুচ্...
3,/kaggle/input/denoiser-for-benchmark/raw/train...,ব্যাটের বেশ দূর থেকে এ সুইং করতেন তিনি,ব্যাটের বেশ দূর থেকে এ সুইং করতেন তিনি
4,/kaggle/input/denoiser-for-benchmark/raw/train...,অধ্যাবধী এ সংগ্রহটি প্রথমশ্রেণীর ক্রিকেটে বিশ্...,অদ্যাবধি এ সংগ্রহটি প্রথমশ্রেণীর ক্রিকেটে বিশ্...
...,...,...,...
20708,/kaggle/input/denoiser-for-benchmark/raw/slr53...,রাজনৈতিক সমাবেশ,রাজনৈতিক সমাবেশ
20709,/kaggle/input/denoiser-for-benchmark/raw/slr53...,এমন সংসদ সদস্য,এমন সংসদ সদস্য
20710,/kaggle/input/denoiser-for-benchmark/raw/slr53...,ল্বেড কেল,ব্লেড গেল
20711,/kaggle/input/denoiser-for-benchmark/raw/slr53...,তাদের ছেলে মেয়েদের,তাদের ছেলেমেয়েদের


In [9]:
!git clone https://github.com/mushrafi88/asr_bangla.git

Cloning into 'asr_bangla'...
remote: Enumerating objects: 425, done.
remote: Counting objects: 100% (154/154), done.
remote: Compressing objects: 100% (91/91), done.
remote: Total 425 (delta 73), reused 129 (delta 61), pack-reused 271
Receiving objects: 100% (425/425), 167.24 MiB | 22.19 MiB/s, done.
Resolving deltas: 100% (206/206), done.
Updating files: 100% (41/41), done.


In [10]:
sym_spell_word_segmentation = SymSpell(max_dictionary_edit_distance=0, prefix_length=7)
dictionary_path = '/kaggle/working/asr_bangla/data_2022/prothom_alo_word_freq.txt'
sym_spell_word_segmentation.load_dictionary(dictionary_path, 0, 1,separator=" ")

True

In [11]:
sym_spell = SymSpell(max_dictionary_edit_distance=5, prefix_length=7)
sym_spell.load_dictionary(dictionary_path, 0, 1,separator=" ")

True

In [12]:
def word_segmentation(input_term):
    result = sym_spell_word_segmentation.word_segmentation(input_term)
    return result.corrected_string
def lookup(sen):
    words = sen.split()
    l = []
    for wow in words:
        if (word_frequency(wow,'bn',wordlist='large',minimum=0.0) == 0.0):
            s='[MASK]'
            l.append(s)
        else:
            l.append(wow)
    return ' '.join(l)


In [13]:
df['input'] = df['wav2vec2'].parallel_apply(lookup)

In [14]:
def mask_len(sen):
    sen = sen.split()
    return sen.count('[MASK]')

In [15]:
df['mask_list'] = df['input'].parallel_apply(mask_len)

In [16]:
df_unsym = df[df.mask_list == 0]

In [17]:
df_sym = df[df.mask_list > 0]
df_direct = df_sym[df_sym.mask_list == 1]
df_indirect = df_sym[df_sym.mask_list > 1]

In [18]:
df_indirect[df_indirect.mask_list != 2]

,path,wav2vec2,sentence,input,mask_list
6,/kaggle/input/denoiser-for-benchmark/raw/train...,যেতা পরিচালনা করেছেন রাখাল বন্ধুরই পরিচালক ইবন...,যেটা পরিচালনা করেছেন রাখাল বন্ধুর ই পরিচালক ইব...,যেতা পরিচালনা করেছেন রাখাল [MASK] পরিচালক ইবনে...,3
8,/kaggle/input/denoiser-for-benchmark/raw/train...,প্রাথমিকভাবে কমিশন তৃষ প্ছরের বেশি পরাশ মহিলাত...,প্রাথমিকভাবে কমিশন ত্রিশ বছরের বেশি বয়সী মহিলা...,প্রাথমিকভাবে কমিশন [MASK] [MASK] বেশি [MASK] [...,4
12,/kaggle/input/denoiser-for-benchmark/raw/train...,এর অধীনস্থ পুরোলিয়ার মানভূম সমবায় দূর্ধ উৎপাদক...,এর অধীনস্থ পুরুলিয়ার মানভূম সমবায় দুগ্ধ উৎপাদক...,এর অধীনস্থ [MASK] মানভূম সমবায় [MASK] উৎপাদক [...,3
23,/kaggle/input/denoiser-for-benchmark/raw/train...,শান্তির পথে চিত্রটতেতিনি মুহাত্মাগান দিকে গেছে...,শান্তির পথে চিত্রটিতে তিনি মহাত্মা গান্ধীকে এঁ...,শান্তির পথে [MASK] [MASK] দিকে গেছেন কটি হিন্দ...,3
24,/kaggle/input/denoiser-for-benchmark/raw/train...,সংক্কৃত বন্ণ প্রাণীর ক্ষেত্রে সকলকে সতর্কতা অব...,সংরক্ষিত বন্যপ্রাণীর ক্ষেত্রে সকলকে সতর্কতা অব...,[MASK] [MASK] প্রাণীর ক্ষেত্রে সকলকে সতর্কতা [...,3
...,...,...,...,...,...
20430,/kaggle/input/denoiser-for-benchmark/raw/slr53...,হাইকোর্ড বিভাগে প্রতিবেদন দাকিল কর্বে,হাইকোর্ট বিভাগে প্রতিবেদন দাখিল করবে,[MASK] বিভাগে প্রতিবেদন [MASK] [MASK],3
20507,/kaggle/input/denoiser-for-benchmark/raw/slr53...,আন্তর্জাতিক মাতরিভাষা দিবস কে সামনের এক্ষে বাং...,আন্তর্জাতিক মাতৃভাষা দিবসকে সামনে রেখে বাংলাদে...,আন্তর্জাতিক [MASK] দিবস কে সামনের [MASK] বাংলা...,4
20602,/kaggle/input/denoiser-for-benchmark/raw/slr53...,পার্বত্ত এলাকা থেকে নিরাপত্যা বাহিনীর দুইষ আটত...,পার্বত্য এলাকা থেকে নিরাপত্তা বাহিনীর টি ক্যাম...,[MASK] এলাকা থেকে [MASK] বাহিনীর [MASK] [MASK]...,4
20603,/kaggle/input/denoiser-for-benchmark/raw/slr53...,মাদিন গুয়ায় আসদেন,মাথিন কুয়ায় আসতেন,[MASK] [MASK] [MASK],3


In [19]:
df.mask_list

0        1
1        0
2        0
3        0
4        1
        ..
20708    0
20709    0
20710    1
20711    0
20712    3
Name: mask_list, Length: 20713, dtype: int64

## For single mask 

In [20]:
error_sen = df_direct['input'].to_list()

In [21]:
len(error_sen)

6834

In [22]:
error_sen[:2]

['ছানার সাথে চিনি বা [MASK] ছাঁচে ফেলে সন্দেশ প্রস্তুত করা হয়ে থাকে',
 '[MASK] এ সংগ্রহটি প্রথমশ্রেণীর ক্রিকেটে বিশ্ব রেকর্ড হিসেবে স্বীকৃত']

In [23]:
chunked_list = list()
chunk_size = 1024
for i in range(0,len(error_sen),chunk_size):
    chunked_list.append(error_sen[i:i+chunk_size])

In [24]:
#chunked_list[0]

In [25]:
inference = []
for batch in chunked_list:
    batch = tokenizer(batch,truncation=True,padding=True, return_tensors="pt").to("cuda")
    translated = model.generate(**batch)
    inference += tokenizer.batch_decode(translated,skip_special_tokens=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [26]:
len(inference)

6834

In [27]:
inference[:3]

['শর্ষে', 'টিটোয়েন্টিতে', 'রেস্তোরাঁ']

In [28]:
len(df_direct)

6834

In [29]:
df_direct['wav2vec2_t5_masked'] = inference

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [30]:
df_direct.sample(5)

,path,wav2vec2,sentence,input,mask_list,wav2vec2_t5_masked
4490,/kaggle/input/denoiser-for-benchmark/raw/train...,প্রথমে এ বিষয়ে কিছু বই ও রচনা পাঠের পর ওয়ালেস ...,প্রথমে এ বিষয়ক কিছু বই ও রচনা পাঠের পর ওয়ালেস ...,প্রথমে এ বিষয়ে কিছু বই ও রচনা পাঠের পর ওয়ালেস ...,1,এ
18046,/kaggle/input/denoiser-for-benchmark/raw/slr53...,বিষ্ণতির মূর্তির,বিষ্ণু ত্রিমূর্তির,[MASK] মূর্তির,1,টি
7206,/kaggle/input/denoiser-for-benchmark/raw/train...,পালক চকচকে নীল বা কালো সাধারণত নিচের চেয়ে উপরে...,পালক চকচকে নীল বা কালো সাধারণত নিচের চেয়ে উপরে...,পালক চকচকে নীল বা কালো সাধারণত নিচের চেয়ে উপরে...,1,থেকে
4549,/kaggle/input/denoiser-for-benchmark/raw/train...,তিনি ধনসম্পদ সংগ্রহ করার জন্য তার অবস্থানের অপ...,তিনি ধনসম্পদ সংগ্রহ করার জন্য তার অবস্থানের অপ...,তিনি ধনসম্পদ সংগ্রহ করার জন্য তার অবস্থানের [M...,1,কাজ
262,/kaggle/input/denoiser-for-benchmark/raw/train...,তার নিয়গে তিনি আন্তর্জাতিক রেলওয়ে ইউনিয়নের সভা...,তাঁর নিয়োগে তিনি আন্তর্জাতিক রেলওয়ে ইউনিয়নের স...,তার [MASK] তিনি আন্তর্জাতিক রেলওয়ে ইউনিয়নের সভ...,1,সঙ্গে


In [31]:
df_direct.reset_index(drop=True)

,path,wav2vec2,sentence,input,mask_list,wav2vec2_t5_masked
0,/kaggle/input/denoiser-for-benchmark/raw/train...,ছানার সাথে চিনি বা গুর্মীশিয়ে ছাঁচে ফেলে সন্দে...,ছানার সাথে চিনি বা গুড় মিশিয়ে ছাঁচে ফেলে সন্দে...,ছানার সাথে চিনি বা [MASK] ছাঁচে ফেলে সন্দেশ প্...,1,শর্ষে
1,/kaggle/input/denoiser-for-benchmark/raw/train...,অধ্যাবধী এ সংগ্রহটি প্রথমশ্রেণীর ক্রিকেটে বিশ্...,অদ্যাবধি এ সংগ্রহটি প্রথমশ্রেণীর ক্রিকেটে বিশ্...,[MASK] এ সংগ্রহটি প্রথমশ্রেণীর ক্রিকেটে বিশ্ব ...,1,টিটোয়েন্টিতে
2,/kaggle/input/denoiser-for-benchmark/raw/train...,সমস্ত শাহতাল ভাইদেরকে নমস্কার,সমস্ত সাঁওতাল ভাইদের কে নমস্কার,সমস্ত [MASK] ভাইদেরকে নমস্কার,1,রেস্তোরাঁ
3,/kaggle/input/denoiser-for-benchmark/raw/train...,এই ক্ষেত্রে সম্প্রচার টাউর্টি হল ট্রান্সমিটার ...,এই ক্ষেত্রে সম্প্রচার টাওয়ারটি হলো ট্রান্সমিটা...,এই ক্ষেত্রে সম্প্রচার [MASK] হল ট্রান্সমিটার র...,1,করা
4,/kaggle/input/denoiser-for-benchmark/raw/train...,উৎসবের সময় মে রা বিচিত্র প্রশাক পড়ে,উৎসবের সময় মেয়েরা বিচিত্র পোশাক পরে,উৎসবের সময় মে রা বিচিত্র [MASK] পড়ে,1,
...,...,...,...,...,...,...
6829,/kaggle/input/denoiser-for-benchmark/raw/slr53...,আট দসমিক সাতান্ন বৈ শতাংশ,দশমিক শতাংশ,আট [MASK] সাতান্ন বৈ শতাংশ,1,
6830,/kaggle/input/denoiser-for-benchmark/raw/slr53...,চিকিৎশকের দেখা পাবেন,চিকিৎসকের দেখা পাবেন,[MASK] দেখা পাবেন,1,এ
6831,/kaggle/input/denoiser-for-benchmark/raw/slr53...,দ্রাগন মুখে এক লোক,ড্রাগনমুখো এক লোক,[MASK] মুখে এক লোক,1,রেস্তোরাঁর
6832,/kaggle/input/denoiser-for-benchmark/raw/slr53...,আমার সমস্ত অহনকার,আমার সমস্ত অহংকার,আমার সমস্ত [MASK],1,


In [32]:
#df.to_parquet('benchmark_2.parquet',index=False)

## df_indirect

In [33]:
from more_itertools import locate
def find_indices(check,item):
    indices = locate(check, lambda x: x == item)
    return list(indices)

In [34]:
df_indirect['wav2vec2'][150]

'রতটি পাক্ষা আকৃতির এবং এর দক্ষিণাংশ বিস্তৃত হলেও উত্তরাশ ক্রমাণ্যয় সংপৃর্ণ হয়েছে'

In [35]:
df_indirect['input'][150]

'[MASK] [MASK] আকৃতির এবং এর দক্ষিণাংশ বিস্তৃত হলেও [MASK] [MASK] [MASK] হয়েছে'

In [36]:
df_indirect = df_indirect.reset_index(drop=True)

In [37]:
df_indirect

,path,wav2vec2,sentence,input,mask_list
0,/kaggle/input/denoiser-for-benchmark/raw/train...,ফলশ্রুতিতে নাটাল দলকে পরাজিত করে গারিকাপের শির...,ফলশ্রুতিতে নাটাল দলকে পরাজিত করে কারি কাপের শি...,ফলশ্রুতিতে নাটাল দলকে পরাজিত করে [MASK] [MASK]...,2
1,/kaggle/input/denoiser-for-benchmark/raw/train...,যেতা পরিচালনা করেছেন রাখাল বন্ধুরই পরিচালক ইবন...,যেটা পরিচালনা করেছেন রাখাল বন্ধুর ই পরিচালক ইব...,যেতা পরিচালনা করেছেন রাখাল [MASK] পরিচালক ইবনে...,3
2,/kaggle/input/denoiser-for-benchmark/raw/train...,প্রাথমিকভাবে কমিশন তৃষ প্ছরের বেশি পরাশ মহিলাত...,প্রাথমিকভাবে কমিশন ত্রিশ বছরের বেশি বয়সী মহিলা...,প্রাথমিকভাবে কমিশন [MASK] [MASK] বেশি [MASK] [...,4
3,/kaggle/input/denoiser-for-benchmark/raw/train...,মহোয়াল গাছের রশাল ফুল এদের দারুণ পছন্দ,মহুয়া গাছের রসালো ফুল এদের দারুণ পছন্দ,[MASK] গাছের [MASK] ফুল এদের দারুণ পছন্দ,2
4,/kaggle/input/denoiser-for-benchmark/raw/train...,সহযুদ্ধাদের নিয়ে পাকিস্তানি আক্রমণের যথপও যুক্...,সহযোদ্ধাদের নিয়ে পাকিস্তানি আক্রমণের যথোপযুক্ত...,[MASK] নিয়ে পাকিস্তানি আক্রমণের [MASK] যুক্ত জ...,2
...,...,...,...,...,...
4922,/kaggle/input/denoiser-for-benchmark/raw/slr53...,আইফুন বা আইপেরের মাধ্যমে,আইফোন বা আইপ্যাডের মাধ্যমে,[MASK] বা [MASK] মাধ্যমে,2
4923,/kaggle/input/denoiser-for-benchmark/raw/slr53...,পসিশ মার্কস মঙ্গলবার সকারে নিদ্যন্টের ক্রিড়া ব...,মার্চ মঙ্গলবার সকালে হৃদযন্ত্রের ক্রিয়া বন্ধ হ...,[MASK] মার্কস মঙ্গলবার সকারে [MASK] ক্রিড়া বন্...,2
4924,/kaggle/input/denoiser-for-benchmark/raw/slr53...,টেশ টেস্ট একশট্টেওয়ান্ডে,টেস্ট ওয়ানডে,[MASK] টেস্ট [MASK],2
4925,/kaggle/input/denoiser-for-benchmark/raw/slr53...,সুক্রবারের বিক্ষবে ডাক দেওয়া হয়,শুক্রবারের বিক্ষোভের ডাক দেওয়া হয়,[MASK] [MASK] ডাক দেওয়া হয়,2


In [38]:
df_indirect[df_indirect.mask_list == 0]

,path,wav2vec2,sentence,input,mask_list


In [39]:
masked_sen = df_indirect['input'][20]
find_indices(masked_sen.split(),'[MASK]')

[5, 8, 10, 11]

In [40]:
[x for x in range(2) ]

[0, 1]

In [41]:
df_temp = pd.DataFrame()
temp_dic = {}
for i in tqdm(range(len(df_indirect))):
    masked_sen = df_indirect['input'][i].split() 
    indices = find_indices(masked_sen,'[MASK]')
    temp = []
    for n in range(len(indices)):
        actual_sen = df_indirect['wav2vec2'][i].split()
        actual_sen[indices[n]] = '[MASK]'
        temp.append(" ".join(actual_sen))
    inference = []
    batch = tokenizer(temp,truncation=True,padding=True, return_tensors="pt").to("cuda")
    translated = model.generate(**batch)
    inference += tokenizer.batch_decode(translated,skip_special_tokens=True)
    temp_dic[i] = {'sentence': df_indirect['sentence'][i] ,'wav2vec2_t5_masked':inference}

100%|██████████| 4927/4927 [05:01<00:00, 16.34it/s]


In [42]:
df_temp = df_temp.from_dict(temp_dic,"index")

In [43]:
df_temp = df_temp.reset_index(drop=True)

In [44]:
#df_indirect['input'].astype('str')

In [45]:
type(df_temp.wav2vec2_t5_masked[0])

list

In [46]:
df_indirect = df_indirect.merge(df_temp)

In [47]:
df_indirect

,path,wav2vec2,sentence,input,mask_list,wav2vec2_t5_masked
0,/kaggle/input/denoiser-for-benchmark/raw/train...,ফলশ্রুতিতে নাটাল দলকে পরাজিত করে গারিকাপের শির...,ফলশ্রুতিতে নাটাল দলকে পরাজিত করে কারি কাপের শি...,ফলশ্রুতিতে নাটাল দলকে পরাজিত করে [MASK] [MASK]...,2,"[টিটোয়েন্টি, সঙ্গে]"
1,/kaggle/input/denoiser-for-benchmark/raw/train...,যেতা পরিচালনা করেছেন রাখাল বন্ধুরই পরিচালক ইবন...,যেটা পরিচালনা করেছেন রাখাল বন্ধুর ই পরিচালক ইব...,যেতা পরিচালনা করেছেন রাখাল [MASK] পরিচালক ইবনে...,3,"[ও, রেহমান, রেহমান]"
2,/kaggle/input/denoiser-for-benchmark/raw/train...,প্রাথমিকভাবে কমিশন তৃষ প্ছরের বেশি পরাশ মহিলাত...,প্রাথমিকভাবে কমিশন ত্রিশ বছরের বেশি বয়সী মহিলা...,প্রাথমিকভাবে কমিশন [MASK] [MASK] বেশি [MASK] [...,4,"[টি, ও, , ]"
3,/kaggle/input/denoiser-for-benchmark/raw/train...,মহোয়াল গাছের রশাল ফুল এদের দারুণ পছন্দ,মহুয়া গাছের রসালো ফুল এদের দারুণ পছন্দ,[MASK] গাছের [MASK] ফুল এদের দারুণ পছন্দ,2,"[টি, টি]"
4,/kaggle/input/denoiser-for-benchmark/raw/train...,মহোয়াল গাছের রশাল ফুল এদের দারুণ পছন্দ,মহুয়া গাছের রসালো ফুল এদের দারুণ পছন্দ,[MASK] গাছের [MASK] ফুল এদের দারুণ পছন্দ,2,"[টি, কতগুলো]"
...,...,...,...,...,...,...
5064,/kaggle/input/denoiser-for-benchmark/raw/slr53...,শুন্তি বাবেন শ্রোতারা,শুনতে পাবেন শ্রোতারা,[MASK] [MASK] শ্রোতারা,2,"[রে, ]"
5065,/kaggle/input/denoiser-for-benchmark/raw/slr53...,পসিশ মার্কস মঙ্গলবার সকারে নিদ্যন্টের ক্রিড়া ব...,মার্চ মঙ্গলবার সকালে হৃদযন্ত্রের ক্রিয়া বন্ধ হ...,[MASK] মার্কস মঙ্গলবার সকারে [MASK] ক্রিড়া বন্...,2,"[রেস্তোরাঁ, ও]"
5066,/kaggle/input/denoiser-for-benchmark/raw/slr53...,টেশ টেস্ট একশট্টেওয়ান্ডে,টেস্ট ওয়ানডে,[MASK] টেস্ট [MASK],2,"[টিটোয়েন্টি, ]"
5067,/kaggle/input/denoiser-for-benchmark/raw/slr53...,সুক্রবারের বিক্ষবে ডাক দেওয়া হয়,শুক্রবারের বিক্ষোভের ডাক দেওয়া হয়,[MASK] [MASK] ডাক দেওয়া হয়,2,"[এ, রেস্তোরাঁ]"


In [48]:
def mask_re_indirect(sen,res):
    sen = sen.split()
    indices = find_indices(sen,'[MASK]')
    if len(indices) == len(res):
        for i in range(len(indices)):
            sen[ indices[i] ] = res[i]
    sen = " ".join(sen)
    sen = sen.split()
    return " ".join(sen)

In [49]:
df_indirect['wav2vec2_t5_masked_'] = df_indirect.parallel_apply(lambda x: mask_re_indirect(x.input,x.wav2vec2_t5_masked),axis=1)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [50]:
df_indirect = df_indirect.drop('wav2vec2_t5_masked',axis=1)
df_indirect.rename(columns = {'wav2vec2_t5_masked_':'wav2vec2_t5_masked'}, inplace = True)
df_indirect

,path,wav2vec2,sentence,input,mask_list,wav2vec2_t5_masked
0,/kaggle/input/denoiser-for-benchmark/raw/train...,ফলশ্রুতিতে নাটাল দলকে পরাজিত করে গারিকাপের শির...,ফলশ্রুতিতে নাটাল দলকে পরাজিত করে কারি কাপের শি...,ফলশ্রুতিতে নাটাল দলকে পরাজিত করে [MASK] [MASK]...,2,ফলশ্রুতিতে নাটাল দলকে পরাজিত করে টিটোয়েন্টি স...
1,/kaggle/input/denoiser-for-benchmark/raw/train...,যেতা পরিচালনা করেছেন রাখাল বন্ধুরই পরিচালক ইবন...,যেটা পরিচালনা করেছেন রাখাল বন্ধুর ই পরিচালক ইব...,যেতা পরিচালনা করেছেন রাখাল [MASK] পরিচালক ইবনে...,3,যেতা পরিচালনা করেছেন রাখাল ও পরিচালক ইবনে রেহম...
2,/kaggle/input/denoiser-for-benchmark/raw/train...,প্রাথমিকভাবে কমিশন তৃষ প্ছরের বেশি পরাশ মহিলাত...,প্রাথমিকভাবে কমিশন ত্রিশ বছরের বেশি বয়সী মহিলা...,প্রাথমিকভাবে কমিশন [MASK] [MASK] বেশি [MASK] [...,4,প্রাথমিকভাবে কমিশন টি ও বেশি ভোটাধিকার প্রদানে...
3,/kaggle/input/denoiser-for-benchmark/raw/train...,মহোয়াল গাছের রশাল ফুল এদের দারুণ পছন্দ,মহুয়া গাছের রসালো ফুল এদের দারুণ পছন্দ,[MASK] গাছের [MASK] ফুল এদের দারুণ পছন্দ,2,টি গাছের টি ফুল এদের দারুণ পছন্দ
4,/kaggle/input/denoiser-for-benchmark/raw/train...,মহোয়াল গাছের রশাল ফুল এদের দারুণ পছন্দ,মহুয়া গাছের রসালো ফুল এদের দারুণ পছন্দ,[MASK] গাছের [MASK] ফুল এদের দারুণ পছন্দ,2,টি গাছের কতগুলো ফুল এদের দারুণ পছন্দ
...,...,...,...,...,...,...
5064,/kaggle/input/denoiser-for-benchmark/raw/slr53...,শুন্তি বাবেন শ্রোতারা,শুনতে পাবেন শ্রোতারা,[MASK] [MASK] শ্রোতারা,2,রে শ্রোতারা
5065,/kaggle/input/denoiser-for-benchmark/raw/slr53...,পসিশ মার্কস মঙ্গলবার সকারে নিদ্যন্টের ক্রিড়া ব...,মার্চ মঙ্গলবার সকালে হৃদযন্ত্রের ক্রিয়া বন্ধ হ...,[MASK] মার্কস মঙ্গলবার সকারে [MASK] ক্রিড়া বন্...,2,রেস্তোরাঁ মার্কস মঙ্গলবার সকারে ও ক্রিড়া বন্ধ ...
5066,/kaggle/input/denoiser-for-benchmark/raw/slr53...,টেশ টেস্ট একশট্টেওয়ান্ডে,টেস্ট ওয়ানডে,[MASK] টেস্ট [MASK],2,টিটোয়েন্টি টেস্ট
5067,/kaggle/input/denoiser-for-benchmark/raw/slr53...,সুক্রবারের বিক্ষবে ডাক দেওয়া হয়,শুক্রবারের বিক্ষোভের ডাক দেওয়া হয়,[MASK] [MASK] ডাক দেওয়া হয়,2,এ রেস্তোরাঁ ডাক দেওয়া হয়


In [51]:
def mask_re_direct(sen,res):
    sen = sen.split()
    indices = find_indices(sen,'[MASK]')
    sen[ indices[0] ] = res
    sen = " ".join(sen)
    sen = sen.split()
    return " ".join(sen)

In [52]:
df_direct['wav2vec2_t5_masked_'] = df_direct.parallel_apply(lambda x: mask_re_direct(x.input,x.wav2vec2_t5_masked),axis=1)
df_direct = df_direct.drop('wav2vec2_t5_masked',axis=1)
df_direct.rename(columns = {'wav2vec2_t5_masked_':'wav2vec2_t5_masked'}, inplace = True)
df_direct

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,path,wav2vec2,sentence,input,mask_list,wav2vec2_t5_masked
0,/kaggle/input/denoiser-for-benchmark/raw/train...,ছানার সাথে চিনি বা গুর্মীশিয়ে ছাঁচে ফেলে সন্দে...,ছানার সাথে চিনি বা গুড় মিশিয়ে ছাঁচে ফেলে সন্দে...,ছানার সাথে চিনি বা [MASK] ছাঁচে ফেলে সন্দেশ প্...,1,ছানার সাথে চিনি বা শর্ষে ছাঁচে ফেলে সন্দেশ প্র...
4,/kaggle/input/denoiser-for-benchmark/raw/train...,অধ্যাবধী এ সংগ্রহটি প্রথমশ্রেণীর ক্রিকেটে বিশ্...,অদ্যাবধি এ সংগ্রহটি প্রথমশ্রেণীর ক্রিকেটে বিশ্...,[MASK] এ সংগ্রহটি প্রথমশ্রেণীর ক্রিকেটে বিশ্ব ...,1,টিটোয়েন্টিতে এ সংগ্রহটি প্রথমশ্রেণীর ক্রিকেটে...
7,/kaggle/input/denoiser-for-benchmark/raw/train...,সমস্ত শাহতাল ভাইদেরকে নমস্কার,সমস্ত সাঁওতাল ভাইদের কে নমস্কার,সমস্ত [MASK] ভাইদেরকে নমস্কার,1,সমস্ত রেস্তোরাঁ ভাইদেরকে নমস্কার
20,/kaggle/input/denoiser-for-benchmark/raw/train...,এই ক্ষেত্রে সম্প্রচার টাউর্টি হল ট্রান্সমিটার ...,এই ক্ষেত্রে সম্প্রচার টাওয়ারটি হলো ট্রান্সমিটা...,এই ক্ষেত্রে সম্প্রচার [MASK] হল ট্রান্সমিটার র...,1,এই ক্ষেত্রে সম্প্রচার করা হল ট্রান্সমিটার রেডি...
22,/kaggle/input/denoiser-for-benchmark/raw/train...,উৎসবের সময় মে রা বিচিত্র প্রশাক পড়ে,উৎসবের সময় মেয়েরা বিচিত্র পোশাক পরে,উৎসবের সময় মে রা বিচিত্র [MASK] পড়ে,1,উৎসবের সময় মে রা বিচিত্র পড়ে
...,...,...,...,...,...,...
20695,/kaggle/input/denoiser-for-benchmark/raw/slr53...,আট দসমিক সাতান্ন বৈ শতাংশ,দশমিক শতাংশ,আট [MASK] সাতান্ন বৈ শতাংশ,1,আট সাতান্ন বৈ শতাংশ
20698,/kaggle/input/denoiser-for-benchmark/raw/slr53...,চিকিৎশকের দেখা পাবেন,চিকিৎসকের দেখা পাবেন,[MASK] দেখা পাবেন,1,এ দেখা পাবেন
20699,/kaggle/input/denoiser-for-benchmark/raw/slr53...,দ্রাগন মুখে এক লোক,ড্রাগনমুখো এক লোক,[MASK] মুখে এক লোক,1,রেস্তোরাঁর মুখে এক লোক
20704,/kaggle/input/denoiser-for-benchmark/raw/slr53...,আমার সমস্ত অহনকার,আমার সমস্ত অহংকার,আমার সমস্ত [MASK],1,আমার সমস্ত


In [53]:
df_sym = df_indirect.append(df_direct)

In [54]:
df_sym = df_sym.reset_index(drop=True)

In [55]:
df_sym

,path,wav2vec2,sentence,input,mask_list,wav2vec2_t5_masked
0,/kaggle/input/denoiser-for-benchmark/raw/train...,ফলশ্রুতিতে নাটাল দলকে পরাজিত করে গারিকাপের শির...,ফলশ্রুতিতে নাটাল দলকে পরাজিত করে কারি কাপের শি...,ফলশ্রুতিতে নাটাল দলকে পরাজিত করে [MASK] [MASK]...,2,ফলশ্রুতিতে নাটাল দলকে পরাজিত করে টিটোয়েন্টি স...
1,/kaggle/input/denoiser-for-benchmark/raw/train...,যেতা পরিচালনা করেছেন রাখাল বন্ধুরই পরিচালক ইবন...,যেটা পরিচালনা করেছেন রাখাল বন্ধুর ই পরিচালক ইব...,যেতা পরিচালনা করেছেন রাখাল [MASK] পরিচালক ইবনে...,3,যেতা পরিচালনা করেছেন রাখাল ও পরিচালক ইবনে রেহম...
2,/kaggle/input/denoiser-for-benchmark/raw/train...,প্রাথমিকভাবে কমিশন তৃষ প্ছরের বেশি পরাশ মহিলাত...,প্রাথমিকভাবে কমিশন ত্রিশ বছরের বেশি বয়সী মহিলা...,প্রাথমিকভাবে কমিশন [MASK] [MASK] বেশি [MASK] [...,4,প্রাথমিকভাবে কমিশন টি ও বেশি ভোটাধিকার প্রদানে...
3,/kaggle/input/denoiser-for-benchmark/raw/train...,মহোয়াল গাছের রশাল ফুল এদের দারুণ পছন্দ,মহুয়া গাছের রসালো ফুল এদের দারুণ পছন্দ,[MASK] গাছের [MASK] ফুল এদের দারুণ পছন্দ,2,টি গাছের টি ফুল এদের দারুণ পছন্দ
4,/kaggle/input/denoiser-for-benchmark/raw/train...,মহোয়াল গাছের রশাল ফুল এদের দারুণ পছন্দ,মহুয়া গাছের রসালো ফুল এদের দারুণ পছন্দ,[MASK] গাছের [MASK] ফুল এদের দারুণ পছন্দ,2,টি গাছের কতগুলো ফুল এদের দারুণ পছন্দ
...,...,...,...,...,...,...
11898,/kaggle/input/denoiser-for-benchmark/raw/slr53...,আট দসমিক সাতান্ন বৈ শতাংশ,দশমিক শতাংশ,আট [MASK] সাতান্ন বৈ শতাংশ,1,আট সাতান্ন বৈ শতাংশ
11899,/kaggle/input/denoiser-for-benchmark/raw/slr53...,চিকিৎশকের দেখা পাবেন,চিকিৎসকের দেখা পাবেন,[MASK] দেখা পাবেন,1,এ দেখা পাবেন
11900,/kaggle/input/denoiser-for-benchmark/raw/slr53...,দ্রাগন মুখে এক লোক,ড্রাগনমুখো এক লোক,[MASK] মুখে এক লোক,1,রেস্তোরাঁর মুখে এক লোক
11901,/kaggle/input/denoiser-for-benchmark/raw/slr53...,আমার সমস্ত অহনকার,আমার সমস্ত অহংকার,আমার সমস্ত [MASK],1,আমার সমস্ত


In [56]:
df_unsym['wav2vec2_t5_masked'] = df_unsym.wav2vec2
df_unsym

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,path,wav2vec2,sentence,input,mask_list,wav2vec2_t5_masked
1,/kaggle/input/denoiser-for-benchmark/raw/train...,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,0,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক
2,/kaggle/input/denoiser-for-benchmark/raw/train...,গড়ি মানুষ কেসে মানুষ বলি মনে করে না এবং তুচ্ছত...,গরীব মানুষকে সে মানুষ বলেই মনে করে না এবং তুচ্...,গড়ি মানুষ কেসে মানুষ বলি মনে করে না এবং তুচ্ছত...,0,গড়ি মানুষ কেসে মানুষ বলি মনে করে না এবং তুচ্ছত...
3,/kaggle/input/denoiser-for-benchmark/raw/train...,ব্যাটের বেশ দূর থেকে এ সুইং করতেন তিনি,ব্যাটের বেশ দূর থেকে এ সুইং করতেন তিনি,ব্যাটের বেশ দূর থেকে এ সুইং করতেন তিনি,0,ব্যাটের বেশ দূর থেকে এ সুইং করতেন তিনি
11,/kaggle/input/denoiser-for-benchmark/raw/train...,এরপর সাথে সাথেই এর বিস্তারিত বর্ণনা দেওয়া শুরু...,এরপর সাথে সাথেই এর বিস্তারিত বর্ণনা দেয়া শুরু ...,এরপর সাথে সাথেই এর বিস্তারিত বর্ণনা দেওয়া শুরু...,0,এরপর সাথে সাথেই এর বিস্তারিত বর্ণনা দেওয়া শুরু...
13,/kaggle/input/denoiser-for-benchmark/raw/train...,এই অঞ্চলের অসংখ্য জলপথ ছিল পরিবহনের প্রধান মাধ্যম,এই অঞ্চলের অসংখ্য জলপথ ছিল পরিবহনের প্রধান মাধ্যম,এই অঞ্চলের অসংখ্য জলপথ ছিল পরিবহনের প্রধান মাধ্যম,0,এই অঞ্চলের অসংখ্য জলপথ ছিল পরিবহনের প্রধান মাধ্যম
...,...,...,...,...,...,...
20706,/kaggle/input/denoiser-for-benchmark/raw/slr53...,তিনজন নিহত,তিনজন নিহত,তিনজন নিহত,0,তিনজন নিহত
20707,/kaggle/input/denoiser-for-benchmark/raw/slr53...,একাধিকবার আহবান জানিয়েছেন,একাধিকবার আহ্বান জানিয়েছেন,একাধিকবার আহবান জানিয়েছেন,0,একাধিকবার আহবান জানিয়েছেন
20708,/kaggle/input/denoiser-for-benchmark/raw/slr53...,রাজনৈতিক সমাবেশ,রাজনৈতিক সমাবেশ,রাজনৈতিক সমাবেশ,0,রাজনৈতিক সমাবেশ
20709,/kaggle/input/denoiser-for-benchmark/raw/slr53...,এমন সংসদ সদস্য,এমন সংসদ সদস্য,এমন সংসদ সদস্য,0,এমন সংসদ সদস্য


In [57]:
df_ = df_unsym.append(df_sym)
df_

,path,wav2vec2,sentence,input,mask_list,wav2vec2_t5_masked
1,/kaggle/input/denoiser-for-benchmark/raw/train...,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,0,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক
2,/kaggle/input/denoiser-for-benchmark/raw/train...,গড়ি মানুষ কেসে মানুষ বলি মনে করে না এবং তুচ্ছত...,গরীব মানুষকে সে মানুষ বলেই মনে করে না এবং তুচ্...,গড়ি মানুষ কেসে মানুষ বলি মনে করে না এবং তুচ্ছত...,0,গড়ি মানুষ কেসে মানুষ বলি মনে করে না এবং তুচ্ছত...
3,/kaggle/input/denoiser-for-benchmark/raw/train...,ব্যাটের বেশ দূর থেকে এ সুইং করতেন তিনি,ব্যাটের বেশ দূর থেকে এ সুইং করতেন তিনি,ব্যাটের বেশ দূর থেকে এ সুইং করতেন তিনি,0,ব্যাটের বেশ দূর থেকে এ সুইং করতেন তিনি
11,/kaggle/input/denoiser-for-benchmark/raw/train...,এরপর সাথে সাথেই এর বিস্তারিত বর্ণনা দেওয়া শুরু...,এরপর সাথে সাথেই এর বিস্তারিত বর্ণনা দেয়া শুরু ...,এরপর সাথে সাথেই এর বিস্তারিত বর্ণনা দেওয়া শুরু...,0,এরপর সাথে সাথেই এর বিস্তারিত বর্ণনা দেওয়া শুরু...
13,/kaggle/input/denoiser-for-benchmark/raw/train...,এই অঞ্চলের অসংখ্য জলপথ ছিল পরিবহনের প্রধান মাধ্যম,এই অঞ্চলের অসংখ্য জলপথ ছিল পরিবহনের প্রধান মাধ্যম,এই অঞ্চলের অসংখ্য জলপথ ছিল পরিবহনের প্রধান মাধ্যম,0,এই অঞ্চলের অসংখ্য জলপথ ছিল পরিবহনের প্রধান মাধ্যম
...,...,...,...,...,...,...
11898,/kaggle/input/denoiser-for-benchmark/raw/slr53...,আট দসমিক সাতান্ন বৈ শতাংশ,দশমিক শতাংশ,আট [MASK] সাতান্ন বৈ শতাংশ,1,আট সাতান্ন বৈ শতাংশ
11899,/kaggle/input/denoiser-for-benchmark/raw/slr53...,চিকিৎশকের দেখা পাবেন,চিকিৎসকের দেখা পাবেন,[MASK] দেখা পাবেন,1,এ দেখা পাবেন
11900,/kaggle/input/denoiser-for-benchmark/raw/slr53...,দ্রাগন মুখে এক লোক,ড্রাগনমুখো এক লোক,[MASK] মুখে এক লোক,1,রেস্তোরাঁর মুখে এক লোক
11901,/kaggle/input/denoiser-for-benchmark/raw/slr53...,আমার সমস্ত অহনকার,আমার সমস্ত অহংকার,আমার সমস্ত [MASK],1,আমার সমস্ত


In [58]:
df = pd.read_parquet("/kaggle/input/benchmark-2/benchmark_2.parquet")
df

,path,id,source,audio,sentence,duration,snr,wav2vec2,2gram,3gram,...,wav2vec2_wer,3gram_label,3gram_wer,4gram_label,4gram_wer,wav2vec2_sym,2gram_sym,3gram_sym,4gram_sym,wav2vec2_t5
0,/kaggle/input/denoiser-for-benchmark/raw/train...,30991599,train,noisy,ছানার সাথে চিনি বা গুড় মিশিয়ে ছাঁচে ফেলে সন্দে...,5.544,-44.277168,ছানার সাথে চিনি বা গুর্মীশিয়ে ছাঁচে ফেলে সন্দে...,ছানার সাথে চিনি বা গুড় মিশিয়ে ছাঁচে ফেলে সন্দে...,ছানার সাথে চিনি বা গুড় মিশিয়ে ছাঁচে ফেলে সন্দে...,...,15,"['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', ...",0,"['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', ...",0,ছানার সাথে চিনি বা গুর্মীশিয়ে ছাঁচে ফেলে সন্দে...,ছানার সাথে চিনি বা গুড় মিশিয়ে ছাঁচে ফেলে সন্দে...,ছানার সাথে চিনি বা গুড় মিশিয়ে ছাঁচে ফেলে সন্দে...,ছানার সাথে চিনি বা গুড় মিশিয়ে ছাঁচে ফেলে সন্দে...,ছানার সাথে চিনি বা শৌর্য দিয়ে ছাঁচে ফেলে সন্দ...
1,/kaggle/input/denoiser-for-benchmark/raw/train...,30991687,train,noisy,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,4.032,-46.880779,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,...,0,"['C', 'C', 'C', 'C', 'C', 'C', 'C']",0,"['C', 'C', 'C', 'C', 'C', 'C', 'C']",0,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক
2,/kaggle/input/denoiser-for-benchmark/raw/train...,30991851,train,noisy,গরীব মানুষকে সে মানুষ বলেই মনে করে না এবং তুচ্...,4.320,-56.581566,গড়ি মানুষ কেসে মানুষ বলি মনে করে না এবং তুচ্ছত...,গরিব মানুষ কি সে মানুষ বলে মনে করে না এবং তুচ্...,গরিব মানুষ কি সে মানুষ বলে মনে করে না এবং তুচ্...,...,36,"['I', 'S', 'S', 'C', 'C', 'S', 'C', 'C', 'C', ...",33,"['I', 'S', 'S', 'C', 'C', 'S', 'C', 'C', 'C', ...",33,গড়ি মানুষ কেসে মানুষ বলি মনে করে না এবং তুচ্ছত...,গরিব মানুষ কি সে মানুষ বলে মনে করে না এবং তুচ্...,গরিব মানুষ কি সে মানুষ বলে মনে করে না এবং তুচ্...,গরিব মানুষ কি সে মানুষ বলে মনে করে না এবং তুচ্...,গড়ি মানুষ কেসে মানুষ বলি মনে করে না এবং তুচ্ছ...
3,/kaggle/input/denoiser-for-benchmark/raw/train...,30992090,train,noisy,ব্যাটের বেশ দূর থেকে এ সুইং করতেন তিনি,4.284,-29.025093,ব্যাটের বেশ দূর থেকে এ সুইং করতেন তিনি,ব্যাটের বেশ দূর থেকে সুইং করতেন তিনি,ব্যাটের বেশ দূর থেকে সুইং করতেন তিনি,...,0,"['C', 'C', 'C', 'C', 'D', 'C', 'C', 'C']",12,"['C', 'C', 'C', 'C', 'D', 'C', 'C', 'C']",12,ব্যাটের বেশ দূর থেকে এ সুইং করতেন তিনি,ব্যাটের বেশ দূর থেকে সুইং করতেন তিনি,ব্যাটের বেশ দূর থেকে সুইং করতেন তিনি,ব্যাটের বেশ দূর থেকে সুইং করতেন তিনি,ব্যাটের বেশ দূর থেকে এ সুইং করতেন তিনি
4,/kaggle/input/denoiser-for-benchmark/raw/train...,30992661,train,noisy,অদ্যাবধি এ সংগ্রহটি প্রথমশ্রেণীর ক্রিকেটে বিশ্...,5.760,-32.322632,অধ্যাবধী এ সংগ্রহটি প্রথমশ্রেণীর ক্রিকেটে বিশ্...,অদ্যাবধি এ সংগ্রহটি প্রথম শ্রেণির ক্রিকেটে বিশ...,অদ্যাবধি এ সংগ্রহটি প্রথম শ্রেণির ক্রিকেটে বিশ...,...,33,"['C', 'C', 'C', 'I', 'S', 'C', 'I', 'S', 'C', ...",40,"['C', 'C', 'C', 'I', 'S', 'C', 'I', 'S', 'C', ...",40,অদ্যাবধি এ সংগ্রহটি প্রথমশ্রেণীর ক্রিকেটে বিশ্...,অদ্যাবধি এ সংগ্রহটি প্রথম শ্রেণির ক্রিকেটে বিশ...,অদ্যাবধি এ সংগ্রহটি প্রথম শ্রেণির ক্রিকেটে বিশ...,অদ্যাবধি এ সংগ্রহটি প্রথম শ্রেণির ক্রিকেটে বিশ...,অধ্যাবসায়ী এ সংগ্রহটি প্রথমশ্রেণীর ক্রিকেটে ব...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20708,/kaggle/input/denoiser-for-benchmark/raw/slr53...,056711c865,openslr,noisy,রাজনৈতিক সমাবেশ,3.100,-37.201962,রাজনৈতিক সমাবেশ,রাজনৈতিক সমাবেশ,রাজনৈতিক সমাবেশ,...,0,"['C', 'C']",0,"['C', 'C']",0,রাজনৈতিক সমাবেশ,রাজনৈতিক সমাবেশ,রাজনৈতিক সমাবেশ,রাজনৈতিক সমাবেশ,রাজনৈতিক সমাবেশ
20709,/kaggle/input/denoiser-for-benchmark/raw/slr53...,a3230c3684,openslr,noisy,এমন সংসদ সদস্য,3.300,-36.750912,এমন সংসদ সদস্য,এমন সংসদ সদস্য,এমন সংসদ সদস্য,...,0,"['C', 'C', 'C']",0,"['C', 'C', 'C']",0,এমন সংসদ সদস্য,এমন সংসদ সদস্য,এমন সংসদ সদস্য,এমন সংসদ সদস্য,এমন সংসদ সদস্য
20710,/kaggle/input/denoiser-for-benchmark/raw/slr53...,7f87c122cb,openslr,

In [59]:
df = df.merge(df_,how='inner')
df = df.dropna(how='any')
df

,path,id,source,audio,sentence,duration,snr,wav2vec2,2gram,3gram,...,4gram_label,4gram_wer,wav2vec2_sym,2gram_sym,3gram_sym,4gram_sym,wav2vec2_t5,input,mask_list,wav2vec2_t5_masked
0,/kaggle/input/denoiser-for-benchmark/raw/train...,30991599,train,noisy,ছানার সাথে চিনি বা গুড় মিশিয়ে ছাঁচে ফেলে সন্দে...,5.544,-44.277168,ছানার সাথে চিনি বা গুর্মীশিয়ে ছাঁচে ফেলে সন্দে...,ছানার সাথে চিনি বা গুড় মিশিয়ে ছাঁচে ফেলে সন্দে...,ছানার সাথে চিনি বা গুড় মিশিয়ে ছাঁচে ফেলে সন্দে...,...,"['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', ...",0,ছানার সাথে চিনি বা গুর্মীশিয়ে ছাঁচে ফেলে সন্দে...,ছানার সাথে চিনি বা গুড় মিশিয়ে ছাঁচে ফেলে সন্দে...,ছানার সাথে চিনি বা গুড় মিশিয়ে ছাঁচে ফেলে সন্দে...,ছানার সাথে চিনি বা গুড় মিশিয়ে ছাঁচে ফেলে সন্দে...,ছানার সাথে চিনি বা শৌর্য দিয়ে ছাঁচে ফেলে সন্দ...,ছানার সাথে চিনি বা [MASK] ছাঁচে ফেলে সন্দেশ প্...,1,ছানার সাথে চিনি বা শর্ষে ছাঁচে ফেলে সন্দেশ প্র...
1,/kaggle/input/denoiser-for-benchmark/raw/train...,30991687,train,noisy,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,4.032,-46.880779,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,...,"['C', 'C', 'C', 'C', 'C', 'C', 'C']",0,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,0,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক
2,/kaggle/input/denoiser-for-benchmark/raw/train...,30991851,train,noisy,গরীব মানুষকে সে মানুষ বলেই মনে করে না এবং তুচ্...,4.320,-56.581566,গড়ি মানুষ কেসে মানুষ বলি মনে করে না এবং তুচ্ছত...,গরিব মানুষ কি সে মানুষ বলে মনে করে না এবং তুচ্...,গরিব মানুষ কি সে মানুষ বলে মনে করে না এবং তুচ্...,...,"['I', 'S', 'S', 'C', 'C', 'S', 'C', 'C', 'C', ...",33,গড়ি মানুষ কেসে মানুষ বলি মনে করে না এবং তুচ্ছত...,গরিব মানুষ কি সে মানুষ বলে মনে করে না এবং তুচ্...,গরিব মানুষ কি সে মানুষ বলে মনে করে না এবং তুচ্...,গরিব মানুষ কি সে মানুষ বলে মনে করে না এবং তুচ্...,গড়ি মানুষ কেসে মানুষ বলি মনে করে না এবং তুচ্ছ...,গড়ি মানুষ কেসে মানুষ বলি মনে করে না এবং তুচ্ছত...,0,গড়ি মানুষ কেসে মানুষ বলি মনে করে না এবং তুচ্ছত...
3,/kaggle/input/denoiser-for-benchmark/raw/train...,30992090,train,noisy,ব্যাটের বেশ দূর থেকে এ সুইং করতেন তিনি,4.284,-29.025093,ব্যাটের বেশ দূর থেকে এ সুইং করতেন তিনি,ব্যাটের বেশ দূর থেকে সুইং করতেন তিনি,ব্যাটের বেশ দূর থেকে সুইং করতেন তিনি,...,"['C', 'C', 'C', 'C', 'D', 'C', 'C', 'C']",12,ব্যাটের বেশ দূর থেকে এ সুইং করতেন তিনি,ব্যাটের বেশ দূর থেকে সুইং করতেন তিনি,ব্যাটের বেশ দূর থেকে সুইং করতেন তিনি,ব্যাটের বেশ দূর থেকে সুইং করতেন তিনি,ব্যাটের বেশ দূর থেকে এ সুইং করতেন তিনি,ব্যাটের বেশ দূর থেকে এ সুইং করতেন তিনি,0,ব্যাটের বেশ দূর থেকে এ সুইং করতেন তিনি
4,/kaggle/input/denoiser-for-benchmark/raw/train...,30992661,train,noisy,অদ্যাবধি এ সংগ্রহটি প্রথমশ্রেণীর ক্রিকেটে বিশ্...,5.760,-32.322632,অধ্যাবধী এ সংগ্রহটি প্রথমশ্রেণীর ক্রিকেটে বিশ্...,অদ্যাবধি এ সংগ্রহটি প্রথম শ্রেণির ক্রিকেটে বিশ...,অদ্যাবধি এ সংগ্রহটি প্রথম শ্রেণির ক্রিকেটে বিশ...,...,"['C', 'C', 'C', 'I', 'S', 'C', 'I', 'S', 'C', ...",40,অদ্যাবধি এ সংগ্রহটি প্রথমশ্রেণীর ক্রিকেটে বিশ্...,অদ্যাবধি এ সংগ্রহটি প্রথম শ্রেণির ক্রিকেটে বিশ...,অদ্যাবধি এ সংগ্রহটি প্রথম শ্রেণির ক্রিকেটে বিশ...,অদ্যাবধি এ সংগ্রহটি প্রথম শ্রেণির ক্রিকেটে বিশ...,অধ্যাবসায়ী এ সংগ্রহটি প্রথমশ্রেণীর ক্রিকেটে ব...,[MASK] এ সংগ্রহটি প্রথমশ্রেণীর ক্রিকেটে বিশ্ব ...,1,টিটোয়েন্টিতে এ সংগ্রহটি প্রথমশ্রেণীর ক্রিকেটে...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20850,/kaggle/input/denoiser-for-benchmark/raw/slr53...,056711c865,openslr,noisy,রাজনৈতিক সমাবেশ,3.100,-37.201962,রাজনৈতিক সমাবেশ,রাজনৈতিক সমাবেশ,রাজনৈতিক সমাবেশ,...,"['C', 'C']",0,রাজনৈতিক সমাবেশ,রাজনৈতিক সমাবেশ,রাজনৈতিক সমাবেশ,রাজনৈতিক সমাবেশ,রাজনৈতিক সমাবেশ,রাজনৈতিক সমাবেশ,0,রাজনৈতিক সমাবেশ
20851,/kaggle/input/denoiser-for-benchmark/raw/slr53...,a3230c3684,openslr,noisy,এমন সংসদ সদস্য,3.300,-36.750912,এমন স

In [60]:
df.to_parquet('benchmark_3.parquet',index=False)

In [61]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from random import random, choice
import random
import time

import os
import re
from tqdm import tqdm 

In [62]:
from typing import Dict, List, Tuple, Any, Union


import json

from datasets import load_metric

from tqdm.auto import tqdm
from IPython.display import display, Audio, HTML
from Levenshtein import distance as lev

import warnings 
warnings.filterwarnings('ignore')

In [63]:
def wer(ref, hyp ,debug=False):
    r = ref.split()
    h = hyp.split()
    #costs will holds the costs, like in the Levenshtein distance algorithm
    costs = [[0 for inner in range(len(h)+1)] for outer in range(len(r)+1)]
    # backtrace will hold the operations we've done.
    # so we could later backtrace, like the WER algorithm requires us to.
    backtrace = [[0 for inner in range(len(h)+1)] for outer in range(len(r)+1)]

    OP_OK = 0
    OP_SUB = 1
    OP_INS = 2
    OP_DEL = 3

    DEL_PENALTY=1 # Tact
    INS_PENALTY=1 # Tact
    SUB_PENALTY=1 # Tact
    # First column represents the case where we achieve zero
    # hypothesis words by deleting all reference words.
    for i in range(1, len(r)+1):
        costs[i][0] = DEL_PENALTY*i
        backtrace[i][0] = OP_DEL

    # First row represents the case where we achieve the hypothesis
    # by inserting all hypothesis words into a zero-length reference.
    for j in range(1, len(h) + 1):
        costs[0][j] = INS_PENALTY * j
        backtrace[0][j] = OP_INS

    # computation
    for i in range(1, len(r)+1):
        for j in range(1, len(h)+1):
            if r[i-1] == h[j-1]:
                costs[i][j] = costs[i-1][j-1]
                backtrace[i][j] = OP_OK
            else:
                substitutionCost = costs[i-1][j-1] + SUB_PENALTY # penalty is always 1
                insertionCost    = costs[i][j-1] + INS_PENALTY   # penalty is always 1
                deletionCost     = costs[i-1][j] + DEL_PENALTY   # penalty is always 1

                costs[i][j] = min(substitutionCost, insertionCost, deletionCost)
                if costs[i][j] == substitutionCost:
                    backtrace[i][j] = OP_SUB
                elif costs[i][j] == insertionCost:
                    backtrace[i][j] = OP_INS
                else:
                    backtrace[i][j] = OP_DEL

    # back trace though the best route:
    i = len(r)
    j = len(h)
    numSub = 0
    numDel = 0
    numIns = 0
    numCor = 0
    if debug:
        lines = []
        compares = []
    while i > 0 or j > 0:
        if backtrace[i][j] == OP_OK:
            numCor += 1
            i-=1
            j-=1
            if debug:
                lines.append("C\t" + r[i]+"\t"+h[j])
        elif backtrace[i][j] == OP_SUB:
            numSub +=1
            i-=1
            j-=1
            if debug:
                lines.append("S\t" + r[i]+"\t"+h[j])
        elif backtrace[i][j] == OP_INS:
            numIns += 1
            j-=1
            if debug:
                lines.append("I\t" + "****" + "\t" + h[j])
        elif backtrace[i][j] == OP_DEL:
            numDel += 1
            i-=1
            if debug:
                lines.append("D\t" + r[i]+"\t"+"****")
    return lines

def error_label(ref,hyp):
    li=[]
    lines= wer(ref, hyp,debug=True)
    for line in reversed(lines):
        line = re.sub(r"\t"," ",line)
        line = line.split()
        li.append(line[0])
    return li

def wer_calc(label):
    s=label.count('S')
    d=label.count('D')
    i=label.count('I')
    return int((s+d+i)/len(label)*100)

def masking(arpa):
    sen = arpa.split()
    for words in sen:
        suggestions = sym_spell.lookup(words, Verbosity.CLOSEST,max_edit_distance=0, include_unknown=True)
        for suggestion in suggestions:
            suggestion = str(suggestion)
            suggestion = suggestion.split(',')
            if int(suggestion[2]) == 0:
                mask = '[MASK]'
                sen = [mask if x == words else x for x in sen]
                break
    return " ".join(sen)

def lav_distance(error_word,masked_word):
    return int(lev(error_word, masked_word)/max(len(error_word),len(masked_word))*100)



In [64]:
df['wav2vec2_t5_label'] = df.parallel_apply(lambda x : error_label(x.sentence,x.wav2vec2_t5),axis=1)
df['wer_wav2vec2_t5'] = df.wav2vec2_t5_label.parallel_apply(lambda x: wer_calc(x))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [65]:
df['wav2vec2_t5_mask_label'] = df.parallel_apply(lambda x : error_label(x.sentence,x.wav2vec2_t5_masked),axis=1)
df['wer_wav2vec2_t5_mask'] = df.wav2vec2_t5_mask_label.parallel_apply(lambda x: wer_calc(x))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [66]:
df.columns

Index(['path', 'id', 'source', 'audio', 'sentence', 'duration', 'snr',
       'wav2vec2', '2gram', '3gram', '4gram', '2gram_label', '2gram_wer',
       'wav2vec2_label', 'wav2vec2_wer', '3gram_label', '3gram_wer',
       '4gram_label', '4gram_wer', 'wav2vec2_sym', '2gram_sym', '3gram_sym',
       '4gram_sym', 'wav2vec2_t5', 'input', 'mask_list', 'wav2vec2_t5_masked',
       'wav2vec2_t5_label', 'wer_wav2vec2_t5', 'wav2vec2_t5_mask_label',
       'wer_wav2vec2_t5_mask'],
      dtype='object')

In [67]:
df.to_parquet('benchmark_3.parquet',index=False)